In [1]:
import os
import pandas as pd
import valentine
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma, Cupid
import pprint
import json

In [2]:
# load data from dataset/monitor_specs folder, there's a folder for each source and each file json file contains the specs for a monitor
sources2monitors = {}

for folder in os.listdir('dataset/monitor_specs'):
    monitors = []
    for file in os.listdir('dataset/monitor_specs/' + folder):
        with open('dataset/monitor_specs/' + folder + '/' + file) as f:
            monitor = json.load(f)
            monitors.append(monitor)
            if folder in sources2monitors:
                sources2monitors[folder].append(monitor)
            else:
                sources2monitors[folder] = [monitor]


In [3]:
sources2monitors.get('www.ebay.com').__len__()

4281

In [4]:
# from the dict source2monitors, we get all the unique attributes for each source and count how many times they appear
sources2attributes = {}

for source in sources2monitors:
    attributes = {}
    for monitor in sources2monitors.get(source):
        for attribute in monitor.keys():
            if attribute in attributes:
                attributes[attribute] += 1
            else:
                attributes[attribute] = 1
    sources2attributes[source] = attributes

In [5]:
sources2attributes.get('www.ebay.com')

{'<page title>': 4281,
 'brand': 2148,
 'condition': 2405,
 'display technology': 3247,
 'screen size': 3495,
 'aspect ratio': 2806,
 'brightness': 2173,
 'contrast ratio': 2733,
 'max resolution': 1205,
 'model': 3055,
 'mpn': 2400,
 'response time': 2715,
 'video inputs': 1654,
 'countryregion of manufacture': 125,
 'max. resolution': 1206,
 'system': 1442,
 'upc': 1762,
 'country/region of manufacture': 90,
 'features': 909,
 'adjustable display angle': 269,
 'adjustable display height': 101,
 'audio': 97,
 'backlight technology': 390,
 'brand name': 381,
 'builtin devices': 121,
 'color': 227,
 'color supported': 289,
 'depth': 310,
 'dvi': 149,
 'general information': 159,
 'green compliance certificateauthority': 278,
 'hdcp supported': 220,
 'hdmi': 117,
 'headphone': 40,
 'horizontal viewing angle': 312,
 'input voltage': 148,
 'interfacesports': 171,
 'manufacturer': 580,
 'manufacturer part number': 292,
 'maximum resolution': 508,
 'miscellaneous': 403,
 'number of screens':

In [6]:
# remove attributes that appear less than 10% of the times in the products of a source
for source in sources2attributes:
    for attribute in list(sources2attributes.get(source).keys()):
        if sources2attributes.get(source).get(attribute) < 0.1 * len(sources2monitors.get(source)):
            sources2attributes.get(source).pop(attribute)
    

In [7]:
sources2attributes.get('www.ebay.com')

{'<page title>': 4281,
 'brand': 2148,
 'condition': 2405,
 'display technology': 3247,
 'screen size': 3495,
 'aspect ratio': 2806,
 'brightness': 2173,
 'contrast ratio': 2733,
 'max resolution': 1205,
 'model': 3055,
 'mpn': 2400,
 'response time': 2715,
 'video inputs': 1654,
 'max. resolution': 1206,
 'system': 1442,
 'upc': 1762,
 'features': 909,
 'manufacturer': 580,
 'maximum resolution': 508}

In [8]:
# create a dict source2dataframe where each dataframe contains the attributes of the products of a source
sources2dataframe = {}

for source in sources2attributes:
    attributes = list(sources2attributes.get(source).keys())
    df = pd.DataFrame(columns=attributes)
    for monitor in sources2monitors.get(source):
        row = []
        for attribute in attributes:
            if attribute in monitor:
                row.append(monitor.get(attribute))
            else:
                row.append(None)
        df.loc[len(df)] = row
    sources2dataframe[source] = df

In [9]:
sources2dataframe.get('www.ebay.com').head()

,<page title>,brand,condition,display technology,screen size,aspect ratio,brightness,contrast ratio,max resolution,model,mpn,response time,video inputs,max. resolution,system,upc,features,manufacturer,maximum resolution
0,"CTL Black 17"" LCD Monitor 171LX w Power and VG...",CTL,Used: An item that has been used previously. T...,LCD,"17""",None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,HP Compaq LE1711 17 inch LCD Monitor New Item ...,HP,None,LCD,"17""",5:4,250 cd/mÂ²,1000,1280 x 1024,LE1711,"EM886A8, EM886A8#ABA",5 ms,VGA D-Sub,None,None,None,None,None,None
2,"Dell 24"" Widescreen LED Backlight 1080p Full H...",None,"New: A brand-new, unused, unopened, undamaged ...",LED LCD,"24""",16:9,250 cd/mÂ²,1000,None,E2414H,320-9776,5 ms,DVI-D,1920 x 1080,Desktop,884116123446,None,None,None
3,New Samsung T27D390 Series 3 LED HDTV Monitor ...,Samsung,"New: A brand-new, unused, unopened, undamaged ...",LED LCD,"27""",16:9,None,None,None,None,None,None,None,None,None,None,None,None,None
4,"Acer 17"" Computer Monitor | eBay",Acer,Used: An item that has been used previously. T...,None,"17""",None,None,None,None,None,None,None,None,None,Desktop,None,None,None,None


In [10]:
# load the mediated schema from monitor_mediated_schema.txt
with open('dataset/monitor_mediated_schema.txt', 'r') as f:
    attributes = [line.strip() for line in f]
mediated_schema = pd.DataFrame(columns=attributes)

# load the ground truth from monitor_schema_matching_labelled_data.csv
ground_truth = pd.read_csv('dataset/monitor_schema_matching_labelled_data.csv')
ground_truth.head(2)

,source_attribute_id,target_attribute_name
0,www.planet-computer.it//dimensione visibile or...,screen_size_horizontal
1,www.pc-canada.com//product name,screen_size_diagonal


In [11]:
# create a dict source2ground_truth using a list of pairs (source_attribute_id, target_attribute_name)
sources2ground_truth = {}

for source, target in ground_truth.values:
    # split the string to get the source and the attribute
    source_attribute = source.split('//')
    if source_attribute[0] in sources2ground_truth:
        sources2ground_truth.get(source_attribute[0]).append((source_attribute[1], target))
    else:
        sources2ground_truth[source_attribute[0]] = [(source_attribute[1], target)]

In [12]:
sources2ground_truth.get('www.ebay.com')

[('display format', 'supported_resolution'),
 ('scan format', 'supported_resolution'),
 ('display response time', 'response_time'),
 ('image aspect ratio', 'supported_aspect_ratio'),
 ('dvid', 'dvid_port_quantity'),
 ('dvid', 'has_dvid_port'),
 ('vga input', 'has_vga_port'),
 ('microphone', 'has_microphone'),
 ('support details full contract period', 'warranty_duration'),
 ('number of dvi', 'dvi_port_quantity'),
 ('bluetooth', 'bluetooth'),
 ('screen refresh rate', 'vertical_refresh_rate_max'),
 ('screen refresh rate', 'vertical_refresh_rate_range')]

In [13]:
# Instantiate matcher and run
# Coma requires java to be installed on your machine
matcher = Coma(max_n=10, strategy="COMA_OPT_INST")

output = pd.DataFrame(columns=['source_attribute_id', 'target_attribute_name'])
i = 1

for source in sources2dataframe:
    print(i,"/",len(sources2dataframe), "source:", source)
    df = sources2dataframe.get(source)
    matches = valentine_match(df, mediated_schema, matcher)
    for (_,a1), (_,a2) in matches:
        output.loc[len(output)] = [source+'//'+a1, a2]
    i += 1
        
pd.DataFrame.to_csv(output, 'coma_output_predictions.csv', index=False)

1 / 26 source: ca.pcpartpicker.com
2 / 26 source: catalog.com
3 / 26 source: ce.yikus.com
4 / 26 source: www.best-deal-items.com
5 / 26 source: www.cleverboxes.com
6 / 26 source: www.ebay.com
7 / 26 source: www.getprice.com.au
8 / 26 source: www.hardware-planet.it
9 / 26 source: www.imldirect.it
10 / 26 source: www.itenergy.co.uk
11 / 26 source: www.jrlinton.co.uk
12 / 26 source: www.kingsfieldcomputers.co.uk
13 / 26 source: www.makingbuyingeasy.co.uk
14 / 26 source: www.mediashopuk.com
15 / 26 source: www.mrhightech.com
16 / 26 source: www.nexus-t.co.uk
17 / 26 source: www.odsi.co.uk
18 / 26 source: www.officedepot.com
19 / 26 source: www.ohc24.ch
20 / 26 source: www.pc-canada.com
21 / 26 source: www.pcconnection.com
22 / 26 source: www.planet-computer.it
23 / 26 source: www.shopmania.com
24 / 26 source: www.softwarecity.ca
25 / 26 source: www.vology.com
26 / 26 source: www.xpcpro.com


In [14]:
# Instantiate matcher and run
matcher = Cupid(w_struct=0.4, leaf_w_struct=0.4, th_accept=0.9, parallelism=16)

output = pd.DataFrame(columns=['source_attribute_id', 'target_attribute_name'])
i = 1

for source in sources2dataframe:
    print(i,"/",len(sources2dataframe), " source:", source)
    df = sources2dataframe.get(source)
    matches = valentine_match(df, mediated_schema, matcher)
    for (_,a1), (_,a2) in matches:
        output.loc[len(output)] = [source+'//'+a1, a2]
    i += 1
    
    pd.DataFrame.to_csv(output, 'cupid_output_predictions.csv', index=False)

1 / 26  source: ca.pcpartpicker.com
2 / 26  source: catalog.com
3 / 26  source: ce.yikus.com
4 / 26  source: www.best-deal-items.com
5 / 26  source: www.cleverboxes.com
6 / 26  source: www.ebay.com
7 / 26  source: www.getprice.com.au
8 / 26  source: www.hardware-planet.it
9 / 26  source: www.imldirect.it
10 / 26  source: www.itenergy.co.uk
11 / 26  source: www.jrlinton.co.uk
12 / 26  source: www.kingsfieldcomputers.co.uk
13 / 26  source: www.makingbuyingeasy.co.uk
14 / 26  source: www.mediashopuk.com
